In [2]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json

def get_ticker(inputs):
    company_name = inputs["company_name"] #inputs에는 {"company_name": "Apple"} 이런식으로 들어옴
    ddg = DuckDuckGoSearchAPIWrapper()
    return ddg.run(f"Ticker symbol of {company_name}")
    # ticker = "AAPL"
    # return {"ticker": ticker}

def get_income_statement(inputs):
    ticker = inputs["ticker"]
    print(ticker)
    stock = yfinance.Ticker(ticker)
    # return stock.income_stmt #pandas dataframe
    return json.dumps(stock.income_stmt.to_json()) #json string

def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())

def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json()) #json string


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

# a = get_income_statement({"ticker": "AAPL"})
# print(type(a))
# print(a)


In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from openai import OpenAI

client = OpenAI()

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview",
#     tools=functions,
# )

assistant_id='asst_gkcE5OgmIiWFufkYVqy1CVRk'


In [73]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Apple stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_8gI4b6fEl8gzu0beY3QFVBfm', created_at=1737858602, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [128]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
print(run)

Run(id='run_l9EWRJowTJMGCyTGEHbyhKBV', assistant_id='asst_gkcE5OgmIiWFufkYVqy1CVRk', cancelled_at=None, completed_at=None, created_at=1737860354, expires_at=1737860954, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_8gI4b6fEl8gzu0beY3QFVBfm', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=Functio

In [119]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )

def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages) #list 매소드를 사용하기 위해 객체를 list로 변환
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outpus
    )


In [131]:
resultstatus = get_run(run.id, thread.id).status
print(resultstatus)

completed


In [130]:
messages = get_messages(thread.id)
print(messages)

user: I want to know if the Apple stock is a good buy
assistant: To determine if Apple Inc. (ticker symbol: AAPL) stock is a good buy, we need to consider several aspects of the company's financial health and stock performance. Based on the data retrieved, we can analyze the following:

1. Income Statement: Apple's most recent income statement shows a net income of $93.736 billion with total revenue of $391.035 billion. The company's profitability is reflected in its significant operating income and gross profit. Consistently high net income and revenue indicate that the company is financially healthy and profitable, which is a positive sign for investors.

2. Balance Sheet: Apple's total assets amount to $364.98 billion, with a common stock equity of $56.95 billion. The company has $106.629 billion in total debt and $76.686 billion in net debt (total debt minus cash and equivalents). Despite having debt, Apple's assets, particularly its investments and advances, strong equity position

In [85]:
submit = get_tool_outputs(run.id, thread.id)
print(submit)

Calling function: get_ticker with arg {"company_name":"Apple"}
[{'output': {'ticker': 'AAPL'}, 'tool_call_id': 'call_z0FpeEd2PSnNdutQXjbTo690'}]


In [3]:
# submit = submit_tool_outputs(run.id, thread.id)
# print(submit)

ticker = get_ticker({"company_name": "Apple"})
print(ticker)
# print(type(ticker))
# print({"ticker": "AAPL"})
# print(type({"ticker": "AAPL"}))
# # print(ticker["ticker"])
# # print(type(ticker))
# # print(type({"ticker": "AAPL"}))

# result1 = get_income_statement(ticker)
# # result2 = get_income_statement({"ticker": "AAPL"})
# print(result1)
# # print(result2)

AttributeError: 'DuckDuckGoSearchAPIWrapper' object has no attribute 'text'

In [26]:
send_message(thread.id, "please go ahead!")

Message(id='msg_UDQKSNK8Ttzqpwv7PFB3W4FU', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='please go ahead!'), type='text')], created_at=1737856998, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_vjAs6q82OZc60auzVaULawnh')